In [1]:
### Notebook for plotting graphics from polygons

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
from bs4 import BeautifulSoup




##### Goals for notebook

create polygons, and rotate to the same orientation.

theory: grab the linestring of one of the foul line measurements. should just be first 2 coordinates of either polygon - check the distance to make sure they are viable lines

measure the angle from the line string to a reference, either 180 of 90 d . create a factor to rotate the plot so reference line is 45 degrees areound the home plate point  then plot


In [2]:
#### Load data from kml file exported by Google Earth

file_path = ('data/kml/ballparks.kml')

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
# layers = soup.Document.Folder.Folder
# polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])


## Loop through the folders and extract the data
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Cleaning
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop any rows with empty fields
df = df[(df != 0).all(1)]

# ## remove any numberic characters and . from field names
# # There shouldn't be any anymore because of the validation I did before exporting kml
# df['field'] = df['field'].str.replace(r'\d+', '')

## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

df['foul_poly'] = df['foul'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})
df['fop_poly'] = df['fop'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})



C:\Users\Smitty\AppData\Local\Temp\ipykernel_10324\1836174812.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Smitty\AppData\Local\Temp\ipykernel_10324\1836174812.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Smitty\AppData\Local\Temp\ipykernel_10324\1836174812.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Smitty\AppData\Local\Temp\ipykernel_10324\1836174812.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Smitty\AppData\Loca

In [3]:
df.head()

,field,foul,fop,level,foul_poly,fop_poly
0,Adrian College,"-84.0697145,41.901861,0 -84.0703958,41.9026485...","-84.0697145,41.901861,0 -84.0687248,41.9023461...",college,"{'type': 'Polygon', 'coordinates': [[(-84.0697...","{'type': 'Polygon', 'coordinates': [[(-84.0697..."
1,Adrian Lenawee Christian HS,"-84.08374689999999,41.9087669,0 -84.0841116999...","-84.08374689999999,41.9087669,0 -84.0826392,41...",high_school,"{'type': 'Polygon', 'coordinates': [[(-84.0837...","{'type': 'Polygon', 'coordinates': [[(-84.0837..."
2,Adrian Lenawee Christian HS- replot,"-84.08374923296931,41.90876073773172,0 -84.084...","-84.08263859810114,41.90903328830586,0 -84.082...",high_school,"{'type': 'Polygon', 'coordinates': [[(-84.0837...","{'type': 'Polygon', 'coordinates': [[(-84.0826..."
3,Albion College,"-84.7373745,42.241567,0 -84.7373704,42.2424775...","-84.7373745,42.241567,0 -84.73614999999999,42....",college,"{'type': 'Polygon', 'coordinates': [[(-84.7373...","{'type': 'Polygon', 'coordinates': [[(-84.7373..."
4,Aldai Stevenson HS,"-83.01445630000001,42.5871203,0 -83.0156645999...","-83.01445630000001,42.5871203,0 -83.014231,42....",high_school,"{'type': 'Polygon', 'coordinates': [[(-83.0144...","{'type': 'Polygon', 'coordinates': [[(-83.0144..."


In [4]:
############ TRANSFORM ###############


# get home plate coords for each foul and fop
## this will be the first coordinate in the list of coordinates

df['foul_hp'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[0:2])))
df['fop_hp'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[0:2])))

# FLIP THE COORDINATES
df['foul_hp'] = df['foul_hp'].apply(lambda x: (x[1], x[0]))
df['fop_hp'] = df['fop_hp'].apply(lambda x: (x[1], x[0]))

## Drop the extra zeros from the coordinates in the foul and fop columns
df['foul'] = df['foul'].apply(lambda x: x.replace(',0', ','))
df['fop'] = df['fop'].apply(lambda x: x.replace(',0', ','))

## get the second coordinate in the list of coordinates
df['foul_2nd'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[3:5])))
df['fop_2nd'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[3:5])))

## Create a line from home plate to the second coordinate
df['foul_line'] = df.apply(lambda x: {'type': 'LineString', 'coordinates': [x['foul_hp'], x['foul_2nd']]}, axis=1)
df['fop_line'] = df.apply(lambda x: {'type': 'LineString', 'coordinates': [x['fop_hp'], x['fop_2nd']]}, axis=1)


df.head()


,field,foul,fop,level,foul_poly,fop_poly,foul_hp,fop_hp,foul_2nd,fop_2nd,foul_line,fop_line
0,Adrian College,"-84.0697145,41.901861, -84.0703958,41.9026485,...","-84.0697145,41.901861, -84.0687248,41.9023461,...",college,"{'type': 'Polygon', 'coordinates': [[(-84.0697...","{'type': 'Polygon', 'coordinates': [[(-84.0697...","(41.901861, -84.0697145)","(41.901861, -84.0697145)","(41.9026485, -84.0704012)","(41.9023461, -84.0688009)","{'type': 'LineString', 'coordinates': [(41.901...","{'type': 'LineString', 'coordinates': [(41.901..."
1,Adrian Lenawee Christian HS,"-84.08374689999999,41.9087669, -84.08411169999...","-84.08374689999999,41.9087669, -84.0826392,41....",high_school,"{'type': 'Polygon', 'coordinates': [[(-84.0837...","{'type': 'Polygon', 'coordinates': [[(-84.0837...","(41.9087669, -84.0837469)","(41.9087669, -84.0837469)","(41.9095844, -84.0841734)","(41.9090424, -84.082702)","{'type': 'LineString', 'coordinates': [(41.908...","{'type': 'LineString', 'coordinates': [(41.908..."
2,Adrian Lenawee Christian HS- replot,"-84.08374923296931,41.90876073773172, -84.0841...","-84.08263859810114,41.90903328830586, -84.0826...",high_school,"{'type': 'Polygon', 'coordinates': [[(-84.0837...","{'type': 'Polygon', 'coordinates': [[(-84.0826...","(41.90876073773172, -84.08374923296931)","(41.90903328830586, -84.08263859810114)","(41.90957624848708, -84.08416036871878)","(41.90907157426523, -84.0826600321213)","{'type': 'LineString', 'coordinates': [(41.908...","{'type': 'LineString', 'coordinates': [(41.909..."
3,Albion College,"-84.7373745,42.241567, -84.7373704,42.2424775,...","-84.7373745,42.241567, -84.73614999999999,42.2...",college,"{'type': 'Polygon', 'coordinates': [[(-84.7373...","{'type': 'Polygon', 'coordinates': [[(-84.7373...","(42.241567, -84.7373745)","(42.241567, -84.7373745)","(42.2424775, -84.7374563)","(42.2415769, -84.7361259)","{'type': 'LineString', 'coordinates': [(42.241...","{'type': 'LineString', 'coordinates': [(42.241..."
4,Aldai Stevenson HS,"-83.01445630000001,42.5871203, -83.01566459999...","-83.01445630000001,42.5871203, -83.014231,42.5...",high_school,"{'type': 'Polygon', 'coordinates': [[(-83.0144...","{'type': 'Polygon', 'coordinates': [[(-83.0144...","(42.5871203, -83.0144563)","(42.5871203, -83.0144563)","(42.5872902, -83.0156069)","(42.5879833, -83.0154809)","{'type': 'LineString', 'coordinates': [(42.587...","{'type': 'LineString', 'coordinates': [(42.587..."


# Start Below

In [19]:
#### Combine the foul and fop polygons into a single Polygon


########## NOT WORKING ##########


from shapely.ops import cascaded_union
from shapely.geometry import Polygon, LineString, Point

# i = 123
boundry = []
new = []

for i in range(len(df)):
    try:
        poly1 = df['foul_poly'][i]
        poly2 = df['fop_poly'][i]
        polygons = [poly1, poly2]

        df['total_poly'][i] = gpd.GeoSeries(unary_union(polygons))

        
        i+=1
    except:
        pass
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 301
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   field      296 non-null    object
 1   foul       296 non-null    object
 2   fop        296 non-null    object
 3   level      296 non-null    object
 4   foul_poly  296 non-null    object
 5   fop_poly   296 non-null    object
 6   foul_hp    296 non-null    object
 7   fop_hp     296 non-null    object
 8   foul_2nd   296 non-null    object
 9   fop_2nd    296 non-null    object
 10  foul_line  296 non-null    object
 11  fop_line   296 non-null    object
dtypes: object(12)
memory usage: 38.2+ KB


In [6]:

# plot the outline of the field boundry

import matplotlib.pyplot as plt

i = 160

boundry = df['boundary'][i]
boundary.plot(facecolor="none", edgecolor='black', lw=0.7)
plt.show()

# boundary.plot(facecolor="none", 
#               edgecolor='black', lw=0.7)
# plt.show()

KeyError: 'boundary'

In [ ]:
import math

def angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    rads = math.atan2(-dy,dx)
    rads %= 2*math.pi
    degs = math.degrees(rads)
    return degs

df['foul_angle'] = df.apply(lambda x: angle(x['foul_hp'][0], x['foul_hp'][1], x['foul_2nd'][0], x['foul_2nd'][1]), axis=1)
df['fop_angle'] = df.apply(lambda x: angle(x['fop_hp'][0], x['fop_hp'][1], x['fop_2nd'][0], x['fop_2nd'][1]), axis=1)

df.head()



In [ ]:
import shapely
from shapely.geometry import shape, Point, Polygon, LineString


## Goal orient all the polygons to the same angle

## Rotate each polygone to the same angle relative to horizontal

## Rotate each line to the same angle relative to horizontal
# i = 120

for i in range(0, len(df)):
    try:
        ### Rotate polygon
        geom = shape(df['boundry'][i])
        angle = 45 - (df['foul_angle'][i])


        # print(geom, angle)
        rot_plot = shapely.affinity.rotate(geom, angle, origin= Point(df[df['foul_hp']]), use_radians=False)
        

        ## add the rotated polygons to the dataframe
        df['boundry'][i] = shapely.geometry.mapping(rot_plot)
        

    

    ## pass on exceptions
    except:
        pass

df.head()
    




# Returns a rotated geometry on a 2D plane.  
# The angle of rotation can be specified in either degrees (default) or   radians by setting use_radians=True. Positive angles are counter-clockwise and   negative are clockwise rotations.  
# The point of origin can be a keyword ‘center’ for the bounding box center   (default), ‘centroid’ for the geometry’s centroid, a Point object or a   coordinate tuple (x0, y0).  
# The affine transformation matrix for 2D rotation is:  
# / cos(r) -sin(r) xoff | sin(r) cos(r) yoff | 0 0 1 /
# where the offsets are calculated from the origin Point(x0, y0)



In [ ]:
#### Make plots of the polygons

import matplotlib.pyplot as plt
import descartes
from descartes import PolygonPatch



In [ ]:
from shapely.geometry import Polygon, Point, LineString
i = 210

plt.rcParams["figure.figsize"] = [7.00, 7.00]
plt.rcParams["figure.autolayout"] = True
polygon1 = Polygon(df['foul_poly'][i]['coordinates'][0])
polygon2 = Polygon(df['fop_poly'][i]['coordinates'][0])

x2, y2 = polygon2.exterior.xy
x, y = polygon1.exterior.xy


## Add text to plot

plt.text(x[0], y[0], df['field'][i], fontsize=12)

plt.plot(x, y, c="red")
plt.plot(x2, y2, c="blue")
plt.show()#

In [ ]:
df.head()

i = 130

plt.rcParams["figure.figsize"] = [7.00, 7.00]
plt.rcParams["figure.autolayout"] = True
polygon1 = Polygon(df['foul_poly'][i]['coordinates'][0])
polygon2 = Polygon(df['fop_poly'][i]['coordinates'][0])

x2, y2 = polygon2.exterior.xy
x, y = polygon1.exterior.xy


## Add text to plot

plt.text(x[0], y[0], df['field'][i], fontsize=12)

plt.plot(x, y, c="red")
plt.plot(x2, y2, c="blue")
plt.show()